In [16]:
from dotenv import load_dotenv
load_dotenv()
import os
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec



In [9]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(
    name="rag", dimension=1536, metric="cosine", 
    spec=ServerlessSpec(cloud="aws", region="us-east-1")
)



In [12]:
import json
data=json.load(open("reviews.json"))
data['reviews']

In [21]:
processed_data = []
client = OpenAI()

for review in data['reviews']:
    response = client.embeddings.create(
        input=review['review'],
        model="text-embedding-3-small"
    )
    embedding = response.data[0].embedding
    processed_data.append({
        "values": embedding,
        "id": review['professor'],
        "metadata": {
            "review":review["review"],
            "subject":review["subject"],
            "stars": review["stars"]
        }
    })

In [22]:
processed_data[0]

{'values': [0.004911996,
  -0.0068521868,
  0.0054230704,
  0.009136249,
  0.03853882,
  0.0016893859,
  0.0031831753,
  0.024544206,
  -0.007950051,
  0.0007500495,
  -0.006725996,
  -0.0038646082,
  -0.015685577,
  -0.045428865,
  -0.005082354,
  0.04303123,
  -0.05229367,
  -0.037857387,
  0.006413672,
  0.0073064757,
  0.028342564,
  0.02345896,
  0.06582137,
  -0.031396393,
  -0.0068143294,
  0.0432079,
  0.058603235,
  0.03321355,
  0.037125476,
  0.038160246,
  0.056987986,
  -0.015042001,
  0.03129544,
  -0.04752364,
  -0.032355446,
  0.054110825,
  -0.009092082,
  0.03333974,
  0.019496555,
  -0.03823596,
  0.0014259615,
  0.020607037,
  -0.020972991,
  -0.003315676,
  0.04113836,
  0.0020663824,
  -0.06380232,
  -0.010997571,
  0.020342035,
  0.032456398,
  -0.046690777,
  0.06228802,
  -0.0016688798,
  -0.02027894,
  -0.031093534,
  -0.011029119,
  0.03967454,
  -0.010644235,
  -0.00040499517,
  -0.03942216,
  0.044949338,
  -0.0010126851,
  0.013843184,
  -0.0050413418,
  -

In [24]:
index = pc.Index('rag')
index.upsert(
    vectors=processed_data,
    namespace="ns1"
)

{'upserted_count': 20}

In [ ]:
index.describe_index_stats()